In [ ]:
import os
import time
from langchain_openai import ChatOpenAI
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from menu_functions import get_menu, detect_language, translate_text
from langchain.schema import Document

def rag_gpt():
    path = "C:/Users/busch/OneDrive/Documents/Fac/M2/UE1 - Advanced programming and data visualization/Advanced Programming/projet/environment/"
    
    load_dotenv(f"{path}.env")
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
    
    llm = ChatOpenAI(model="gpt-3.5-turbo")  
    
    context = False
    question = input("Your question: ")
    question_language = detect_language(question)
    question = translate_text(question, "en")
    
    while question != "Exit":
        if context == True:
            question = input("Your question: ")
            question_language = detect_language(question)
            question = translate_text(question, "en")
        else:
            menus = get_menu(question)
            # Raw text processing
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
            splits = text_splitter.split_text(menus)
            
            # Transforming text into objects 
            documents = [Document(page_content=split) for split in splits]
            
            # Creation of the vectorstore
            vectorstore = InMemoryVectorStore.from_documents(
                documents=documents, embedding=OpenAIEmbeddings()
            )
            retriever = vectorstore.as_retriever()

            # Prompt
            system_prompt = (
                "You are an assistant for question-answering tasks about the menu of university restaurant. "
                "If you don't specify a specific dish, you should always give today's meal or the nearest one. "
                "If i answer you a question about a restaurant and you don't have the contexte of this restaurant, say : 'No context'" 
                "Use the following pieces of retrieved context to answer "
                "the question. If you don't know the answer, say that you "
                "don't know. Use bullet point when it's necessary."
                "\n\n"
                "{context}"
            )
            
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", system_prompt),
                    ("human", "{input}"),
                ]
            )

            question_answer_chain = create_stuff_documents_chain(llm, prompt)
            rag_chain = create_retrieval_chain(retriever, question_answer_chain)
            
            context = True

        if question != "Exit":
            results = rag_chain.invoke({"input": question})
            if results["answer"] == 'No context.' or results["answer"] == 'No context':
                context = False
            else:
                print(translate_text(results["answer"], question_language))


In [3]:
rag_gpt()

Your question:  What is the menu at Esplanade for this diner ?


Menu du mardi 3 décembre 2024 à Esplanade pour le diner :

Menus étudiants 2ème étage
- Cordon bleu, sauce crème maison
- Haricots beurre
- Purée de pommes de terre
 
- Bourguignon de champignons maison
- Purée de pommes de terre

Desserts 2ème étage
- Financier aux myrtilles
- Eclair vanille
- Mini donut

Origines de nos viandes du jour
- Volaille origine France


Your question:  Exit
